In [1]:
#주피터노트북 킨 상태에서 HeidiSQL 수정 등 불가함!
#MariaDB 데이터베이스 뿐만 아니라 테이블도 인코딩 utf8-general-ci로 해줘야됨! 안해놓으면 Incorrect string 에러 발생

import pymysql
import pandas as pd
from urllib.request import urlopen
from datetime import datetime
from bs4 import BeautifulSoup
from threading import Timer
import urllib, pymysql, calendar, time, json

class DBUpdater: #DBUpdater 클래스는 객체가 생성될 때 마리아디비에 접속하고, 소멸될 때 접속을 해제
    def __init__(self):
        """생성자: MariaDB 연결 및 종목코드 딕셔너리 생성"""
        self.conn = pymysql.connect(host = 'localhost', user = 'root', password = 'qwe0611', db='investar', charset='utf8') #일반적으로 한글을 이용하면 오류가 발생하므로, 반드시 utf8로 인코딩 형식 지정
        
        with self.conn.cursor() as curs: #이미 존재하는 테이블에 CREATE TABLE 구문을 추가하면 오류가 발생하므로 IF NOT EXIST 구문을 이용하는 것
            sql = """
            CREATE TABLE IF NOT EXISTS company_info ( 
                code VARCHAR(20),
                company VARCHAR(40),
                last_update DATE,
                PRIMARY KEY (code))
            """
            curs.execute(sql)
            sql = """
            CREATE TABLE IF NOT EXISTS daily_price (
                code VARCHAR(20),
                date DATE,
                open BIGINT(20),
                high BIGINT(20),
                low BIGINT(20),
                close BIGINT(20),
                diff BIGINT(20),
                volume BIGINT(20),
                PRIMARY KEY (code, date))
            """
            curs.execute(sql)
        self.conn.commit()
        self.codes = dict()
        self.update_comp_info()
        
    def __del__(self):
        """소멸자:MariaDB 연결해제"""
        self.conn.close()
        
    def read_krx_code(self):
        """KRX로부터 상장법인목록 파일을 읽어와서 데이터프레임으로 반환"""
       #url = 'https://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13'
       #krx = pd.read_html(url, header=0)[0]
        krx = pd.read_excel('krx_list_500.xlsx') #자료저장 속도 문제로 시총 상위 500개 종목만 추려서 다운
        krx = krx[['종목코드', '회사명']]
        krx = krx.rename(columns={'종목코드':'code', '회사명' : 'company'})
        krx.code = krx.code.map('{:06d}'.format)
        return krx
    
    def update_comp_info(self):
        """종목코드를 company_info 테이블에 업데이트한 후 딕셔너리에 저장"""
        sql = "SELECT * FROM company_info"
        df = pd.read_sql(sql, self.conn) # company_info 테이블을 read_sql()함수로 읽는다
        for idx in range(len(df)):
            self.codes[df['code'].values[idx]] = df['company'].values[idx] # read_sql()로 읽은 데이터프레임을 이용, codes딕셔너리 생성
        with self.conn.cursor() as curs:
            sql = "SELECT max(last_update) FROM company_info" #가장 최근의 날짜 가져오기
            curs.execute(sql)
            rs = curs.fetchone()
            today = datetime.today().strftime('%Y-%m-%d') #strftime은 datetime의 형식을 지정
            
            if rs[0] == None or rs[0].strftime('%Y-%m-%d') < today: #하루에 한번씩 업데이트
                krx = self.read_krx_code()
                for idx in range(len(krx)):
                    code = krx.code.values[idx]
                    company = krx.company.values[idx]
                    sql = f"REPLACE INTO company_info (code, company, last_update) VALUES ('{code}', '{company}', '{today}')"
                    #REPLACE문은 표준 SQL은 아니지만 마리아디비에서 제공되는 SQL문으로 데이터 행이 테이블에 이미 존재했을 때 오류가 나는 INSERT 문의 단점을 보완
                    curs.execute(sql)
                    self.codes[code] = company
                    tmnow = datetime.now().strftime('%Y-%m-%d %H:%M')
                    print(f"[{tmnow}] {idx:04d} REPLACE INTO company_info VALUES ({code}, {company}, {today})")
                self.conn.commit()
            
                    
        
    def read_naver(self, code, company, pages_to_fetch):
        """네이버 금융에서 주식 시세를 읽어서 데이터프레임으로 반환"""
        try:
            url = f"http://finance.naver.com/item/sise_day.nhn?code={code}"
            with urlopen(url) as doc:
                if doc is None:
                    return None
                html = BeautifulSoup(doc,'lxml')
                pgrr = html.find("td", class_ = "pgRR")
                if pgrr is None:
                    return None
                s = str(pgrr.a["href"]).split('=')
                lastpage = s[-1]
            df = pd.DataFrame()
            pages = min(int(lastpage), pages_to_fetch)
            for page in range(1, pages + 1):
                pg_url = '{}&page={}'.format(url,page)
                df = df.append(pd.read_html(pg_url, header = 0)[0])
                tmnow = datetime.now().strftime('%Y-%m-%d %H:%M')
                print('[{}] {} ({}) : {:04d}/{:04d} pages are downloading...'.format(tmnow, company, code, page, pages), end= "\r")
            df = df.rename(columns = {'날짜' : 'date', '종가' : 'close', '전일비' : 'diff', '시가' : 'open', '고가' : 'high', '저가' : 'low', '거래량' : 'volume'})
            df['date'] = df['date'].replace('.','-')
            df = df.dropna()
            df[['close','diff','open','high','low','volume']] = df[['close','diff','open','high','low','volume']].astype(int)
            df = df[['date','open','high','low','close','diff','volume']]
        except Exception as e:
            print('Exception occured :', str(e))
            return None
        return df
    def replace_into_db(self, df, num, code, company):
        """네이버 금융에서 읽어온 주식 시세를 DB에 REPLACE"""
        with self.conn.cursor() as curs:
            for r in df.itertuples():
                sql = f"REPLACE INTO daily_price VALUES ('{code}', '{r.date}', {r.open}, {r.high}, {r.low}, {r.close}, {r.diff}, {r.volume})"
                curs.execute(sql)
            self.conn.commit()
            print("[{}] #{:04d} {} ({}) : {} rows > REPLACE INTO daily_price[OK]".format(datetime.now().strftime('%Y-%m-%d %H:%M'), num+1, company, code, len(df)))
        
    def update_daily_price(self, pages_to_fetch):
        """KRX 상장법인의 주식 시세를 네이버로부터 읽어서 DB에 업데이트"""
        for idx, code in enumerate(self.codes):
            df = self.read_naver(code, self.codes[code], pages_to_fetch)
            if df is None:
                continue
            self.replace_into_db(df, idx, code, self.codes[code])
            
    def execute_daily(self):
        """실행 즉시 매일 오후 다섯시에 daily_price 테이블 업데이트"""
        self.update_comp_info()
        try:
            with open('config.json', 'r') as in_file:
                config = json.load(in_file)
                pages_to_fetch = config['pages_to_fetch']
        except FileNotFoundError:
            with open('config.json', 'w') as out_file:
                pages_to_fetch = 100
                config = {'pages_to_fetch' : 1}
                json.dump(config, out_file)
        self.update_daily_price(pages_to_fetch)
        
        tmnow = datetime.now()
        lastday = calendar.monthrange(tmnow.year, tmnow.month)[1]
        if tmnow.month == 12 and tmnow.day == lastday:
            tmnext = tmnow.replace(year = tmnow.year+1, month=1, day=1, hour = 17, minute = 0, second = 0)
        elif tmnow.day == lastday:
            tmnext = tmnow.replace(month=tmnow.month+1, day=1, hour = 17, minute = 0, second = 0)
        else:
            tmnext = tmnow.replace(day=tmnow.day+1, hour = 17, minute=0, second = 0)
        tmdiff = tmnext - tmnow
        secs = tmdiff.seconds
        
        t = Timer(secs, self.execute_daily)
        print("Waiting for next update ({}) ...".format(tmnext.strftime('%Y-%m-%d %H:%M')))
        t.start()
if __name__=='__main__':
    dbu = DBUpdater()
    dbu.execute_daily()

[2020-09-13 15:06] #0001 동화약품 (000020) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:06] #0002 메리츠화재 (000060) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:06] #0003 삼양홀딩스 (000070) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:06] #0004 하이트진로 (000080) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:06] #0005 유한양행 (000100) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:06] #0006 CJ대한통운 (000120) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:06] #0007 하이트진로홀딩스 (000140) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:06] #0008 두산 (000150) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:06] #0009 대림산업 (000210) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:06] #0010 한국테크놀로지그룹 (000240) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:06] #0011 삼천당제약 (000250) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:06] #0012 한화손해보험 (000370) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:06] #0013 

[2020-09-13 15:07] #0103 한샘 (009240) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:07] #0104 광동제약 (009290) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:07] #0105 태영건설 (009410) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:07] #0106 한올바이오파마 (009420) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:07] #0107 경동나비엔 (009450) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:07] #0108 한국조선해양 (009540) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:07] #0109 한화솔루션 (009830) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:07] #0110 영원무역홀딩스 (009970) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:07] #0111 OCI (010060) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:07] #0112 고려아연 (010130) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:07] #0113 삼성중공업 (010140) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:07] #0114 현대미포조선 (010620) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:07] #0115 아이에스동

[2020-09-13 15:08] #0205 한국가스공사 (036460) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:08] #0206 SK머티리얼즈 (036490) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:08] #0207 SFA반도체 (036540) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:08] #0208 영풍정밀 (036560) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:08] #0209 엔씨소프트 (036570) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:08] #0210 심텍홀딩스 (036710) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:08] #0211 에프에스티 (036810) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:08] #0212 솔브레인홀딩스 (036830) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:08] #0213 주성엔지니어링 (036930) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:08] #0214 LG헬로비전 (037560) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:08] #0215 삼표시멘트 (038500) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:08] #0216 이오테크닉스 (039030) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:08] 

[2020-09-13 15:09] #0306 휠라홀딩스 (081660) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:09] #0307 젬백스 (082270) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:09] #0308 동양생명 (082640) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:09] #0309 비츠로셀 (082920) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:09] #0310 GST (083450) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:09] #0311 비에이치아이 (083650) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:09] #0312 크리스탈 (083790) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:09] #0313 휴온스글로벌 (084110) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:09] #0314 유진테크 (084370) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:09] #0315 랩지노믹스 (084650) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:09] #0316 대상홀딩스 (084690) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:09] #0317 아이티엠반도체 (084850) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:09] #0318 헬릭스미스 (

[2020-09-13 15:10] #0408 엔케이맥스 (182400) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:10] #0409 골프존데카 (183410) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:10] #0410 엔지켐생명과학 (183490) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:10] #0411 종근당 (185750) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:10] #0412 더블유게임즈 (192080) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:10] #0413 쿠쿠홀딩스 (192400) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:10] #0414 슈피겐코리아 (192440) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:10] #0415 드림텍 (192650) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:10] #0416 코스맥스 (192820) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:10] #0417 노바렉스 (194700) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:10] #0418 알테오젠 (196170) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:10] #0419 디오스텍 (196450) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:10] #0420 콜마비앤에이치

[2020-09-13 15:11] #0509 SK5호스팩 (337450) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:11] #0510 SK6호스팩 (340350) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:11] #0511 한화플러스제1호스팩 (340440) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:11] #0512 제이알글로벌리츠 (348950) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:11] #0513 대덕전자 (353200) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:11] #0514 미래에셋대우스팩 5호 (353490) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:11] #0515 솔브레인 (357780) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:11] #0516 크리스탈신소재 (900250) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:11] #0517 엑세스바이오 (950130) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:11] #0518 코오롱티슈진 (950160) : 10 rows > REPLACE INTO daily_price[OK]
[2020-09-13 15:11] #0519 미투젠 (950190) : 10 rows > REPLACE INTO daily_price[OK]
Waiting for next update (2020-09-14 17:00) ...


In [15]:
a = pd.read_excel('krx_list_500.xlsx',header=0)
a = a[['종목코드','회사명']]
a.종목코드 = a.종목코드.map('{:06d}'.format)
a.head(30)

,종목코드,회사명
0,353490,미래에셋대우스팩 5호
1,950190,미투젠
2,348950,제이알글로벌리츠
3,357780,솔브레인
4,332570,와이팜
5,298060,에스씨엠생명과학
6,353200,대덕전자
7,340350,SK6호스팩
8,092190,서울바이오시스
9,204270,제이앤티씨


In [7]:
from datetime import datetime
today = datetime(2020,9,7).strftime('%Y-%m-%d')
today

'2020-09-07'